### LDA

In [1]:
import gensim
import gensim.corpora as corpora
import nltk
from nltk.corpus import stopwords
import pandas as pd 
# import modin.pandas as pd
# import ray
# ray.init(num_cpus=8)
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
import re
import spacy
import string 
from gensim.corpora import Dictionary
from gensim.test.utils import get_tmpfile
import tqdm as tqdm
import glob as glob

In [2]:
import nltk
nltk.download('stopwords')

In [4]:
add_stopwords=['dont','didnt','lol','say','let','wont','ve','do','im','rt','amp']
stop_words = set(stopwords.words('english') + ['AT_USER','URL'] + add_stopwords+stop)

def process_text(text):
    text = str(text)
    text = text.lower() # convert text to lower-case
    text = ''.join([t for t in text if t not in string.punctuation]) # remove punctuation
    text = re.sub( 'http\S+','',text) # remove URLs
    text = re.sub('@\S+', '',text) # remove usernames
    text = re.sub(r'#([^\s]+)', r'\1', text) # remove the # in #hashtag
    text = re.sub("([^\x00-\x7F])+"," ",text) # remove non-ASCII characters
    tkz = nltk.RegexpTokenizer("\\b[\\w-]+\\b")
    text = tkz.tokenize(text)

    return [word for word in text if word not in stop_words and not word.isnumeric() and len(word) >= 2]

In [7]:
data_type = {'created_at':'string','id_str':'string','full_text':'string','user':'object'}
#data=pd.read_csv("/data_8t/lwx/Drugtweets/ivermectin/ivermectin.csv",dtype=data_type, lineterminator='\n',usecols=['id_str','created_at','user','full_text'],low_memory=False, keep_default_na=False)
out_dir='/data_8t/lwx/Drugtweets/graphic/'
files = sorted(glob.glob('/data_8t/lwx/Drugtweets/graphic/'+"*.csv"))

In [6]:
data=pd.read_csv('/data_8t/lwx/Drugtweets/graphic/21April_0.csv',dtype=data_type, lineterminator='\n',usecols=['id_str','created_at','user','full_text'],keep_default_na=False)
for file in tqdm.tqdm(files[1:]):
    dff = pd.read_csv(file,dtype=data_type, lineterminator='\n',usecols=['id_str','created_at','user','full_text'],keep_default_na=False)
    data=pd.concat([data,dff],axis=0)

100%|██████████| 99/99 [01:39<00:00,  1.00s/it]


In [7]:
data

,created_at,id_str,full_text,user
0,Tue Apr 06 00:15:28 +0000 2021,1379226213210787841,I have consumed an unhealthy amount of caffein...,"{'id': 372976945, 'id_str': '372976945', 'name..."
1,Mon Apr 12 14:22:38 +0000 2021,1381613736012701696,@AhhDamnHer Getting the #SARSCoV2 virus u hv 9...,"{'id': 700603156930158593, 'id_str': '70060315..."
2,Thu Apr 15 03:11:58 +0000 2021,1382532122527768576,@ANI @HemantSorenJMM AGAR JHARKHAND BOARD CANC...,"{'id': 1062589313404338176, 'id_str': '1062589..."
3,Thu Apr 22 03:34:19 +0000 2021,1385074461674819585,COVID-19 and APA *LA positivity in COVID-19 ...,"{'id': 1623493783, 'id_str': '1623493783', 'na..."
4,Fri Apr 16 04:01:18 +0000 2021,1382906924124012544,@gathertonourish @Jo_Brant In Quebec and USA I...,"{'id': 899809505101459456, 'id_str': '89980950..."
...,...,...,...,...
19814,Tue Sep 08 10:34:52 +0000 2020,1303280635822379009,@danwellsracing Link between vit D deficiency ...,"{'id': 515933811, 'id_str': '515933811', 'name..."
19815,Sun Sep 13 00:55:08 +0000 2020,1304946684477104129,"@lifebiomedguru From the Drs homepage ....""we ...","{'id': 1290261687367426049, 'id_str': '1290261..."
19816,Wed Sep 09 13:20:11 +0000 2020,1303684628876853248,@CBSNews Flu season approaching. And fall alle...,"{'id': 1576242038, 'id_str': '1576242038', 'na..."
19817,Tue Sep 08 00:04:28 +0000 2020,1303121994553188353,@SisyphusInvest1 Also @BuckQuivas - listened t...,"{'id': 1193383563967782912, 'id_str': '1193383..."


In [8]:
data['pre-processed'] = data['full_text'].apply(process_text)
data_words = data['pre-processed'].to_list()

In [9]:
print(data_words[:2])

[['consumed', 'unhealthy', 'amount', 'caffeine', 'trying', 'stay', 'awake', 'work', 'post', 'vaccine', 'xx'], ['ahhdamnher', 'getting', 'sarscov2', 'virus', 'hv', 'recovery', '998older', 'plus', 'obesity', 'hi', 'blood', 'cancer', 'shd', 'protected', 'take', 'ivermectin']]


In [10]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=50, threshold=150) # higher threshold fewer phrases.

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

# See bigram example
print(bigram_mod[bigram_mod[data_words[0]]])

['consumed', 'unhealthy', 'amount', 'caffeine', 'trying', 'stay', 'awake', 'work', 'post', 'vaccine', 'xx']


In [11]:
# %pip uninstall spacy
# %pip install -i https://pypi.douban.com/simple spacy==2.2.2

In [12]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
    
# Form Bigrams
data_words_bigrams = make_bigrams(data_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [ ]:
import numpy 
numpy.random.seed(1)
def remove_stopwords(data_lemmatized):
    return [[word for word in doc if word not in stop_words and len(word) > 2] for doc in data_lemmatized]
b=remove_stopwords(data_lemmatized[:])
data_lemmatized=remove_stopwords(data_lemmatized)
data_lemmatized

In [20]:
import numpy as np
data_lemmatized=np.array(data_lemmatized)
np.save('data_lemmatized.npy',data_lemmatized) # 保存为.npy格式
a=np.load('data_lemmatized.npy',allow_pickle=True)
a=a.tolist()
data_lemmatized=a

In [13]:
# preserve tokens
tokens = pd.DataFrame({'token':data_lemmatized})
tokens.to_csv('total_tokens.csv')

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
#Gensim 为文档中的每个单词创建一个唯一的 id。上面显示的生成语料库是 (word_id, word_frequency) 的映射。这被用作 LDA 模型的输入

In [14]:
#corpora.MmCorpus.serialize('total_corpuse.mm', corpus)  # 将生成的语料保存成MM文件
corpus = corpora.MmCorpus('total_corpuse.mm')  # 加载

In [15]:
#id2word.save_as_text("totalid2word.txt")#保存生成的dictionary
id2word = Dictionary.load_from_text("totalid2word.txt")#加载生成的dictionary

In [23]:
data_lemmatized=pd.read_csv('total_tokens.csv').token.tolist()#加载词形还原后的分词

#### 将主题打印 计算coherence score

In [11]:
# import numpy as np
# a=np.load('data_lemmatizediver.npy',allow_pickle=True)
# a=a.tolist()
# data_lemmatized=a
# corpus = corpora.MmCorpus('total_corpuseiver.mm')  # 加载
# id2word = Dictionary.load_from_text("totalid2wordiver.txt")#加载生成的dictionary

In [5]:
# #model=gensim.models.ldamodel.LdaModel.load("ldarem_8_184428.model")
# model=gensim.models.ldamodel.LdaModel.load("rem7da.model")
# # Compute Coherence Score
# coherence_model_lda = CoherenceModel(model=model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3668129664017141


In [10]:
# #model=gensim.models.ldamodel.LdaModel.load("ldazinc_4_125362.model")
# # Print the Keyword in the 10 topics
# from pprint import pprint
# pprint(model.print_topics())
# doc_lda = model[corpus]

[(0,
  '0.101*"covid19" + 0.090*"treatment" + 0.044*"study" + 0.039*"trial" + '
  '0.029*"evidence" + 0.023*"effective" + 0.020*"treat" + 0.019*"clinical" + '
  '0.017*"patient" + 0.014*"datum"'),
 (1,
  '0.053*"vaccine" + 0.040*"people" + 0.028*"cure" + 0.028*"treatment" + '
  '0.027*"effective" + 0.023*"safe" + 0.023*"covid" + 0.015*"save" + '
  '0.015*"horse" + 0.014*"prove"'),
 (2,
  '0.024*"zinc" + 0.022*"vitamin" + 0.019*"virus" + 0.017*"watch" + '
  '0.015*"hydroxychloroquinezinc" + 0.013*"vitro_outpatient" + 0.012*"flu" + '
  '0.012*"protect" + 0.012*"health" + 0.012*"parasitic"'),
 (3,
  '0.067*"doctor" + 0.048*"patient" + 0.032*"day" + 0.031*"treat" + '
  '0.025*"treatment" + 0.022*"covid" + 0.019*"hospital" + 0.017*"start" + '
  '0.017*"symptom" + 0.016*"die"'),
 (4,
  '0.042*"virus" + 0.026*"kill" + 0.020*"study" + 0.016*"death" + 0.015*"hour" '
  '+ 0.014*"country" + 0.014*"reduce" + 0.012*"rate" + 0.011*"risk" + '
  '0.011*"low"'),
 (5,
  '0.112*"drug" + 0.042*"treat" + 0

In [10]:
# # Visualize the topics
# import pyLDAvis.gensim_models
# from gensim import corpora
# from gensim.models import LdaModel
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim_models.prepare(model, corpus, id2word)
# vis

/home/lwx/.conda/envs/lwx/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/home/lwx/.conda/envs/lwx/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/lwx/.conda/envs/lwx/lib/python3.8/site-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/home/lwx/.conda/envs/lwx/lib/python3.8/site-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/home/lwx/.conda/envs/lwx/lib/pyth

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.055070  0.297684       1        1  28.748206
4     -0.170578 -0.186220       2        1  22.727452
3     -0.204969  0.121529       3        1  17.823340
0     -0.022141 -0.155113       4        1  16.983150
2      0.342619 -0.077880       5        1  13.717852, topic_info=            Term          Freq         Total Category  logprob  loglift
9267     covid19  32687.000000  32687.000000  Default  30.0000  30.0000
30563    patient  36764.000000  36764.000000  Default  29.0000  29.0000
44027  treatment  31291.000000  31291.000000  Default  28.0000  28.0000
2889     approve  15821.000000  15821.000000  Default  27.0000  27.0000
13340       drug  34228.000000  34228.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
13340       drug   7098.457375  34228.355784   Topic5  -3.2076   0.4133
26860      month   1318.461089   2196.213646   Topic5  -4.8910   1.4762
9267     covid19   2123.137939  32687.013759   Topic5  -4.4145  -0.7476
31244     pharma    995.580688   1780.053856   Topic5  -5.1719   1.4054
38646      share    926.987731   1568.524684   Topic5  -5.2433   1.4605

[242 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
777        4  0.999372          access
1103       2  0.999305  administration
1136       3  0.999650           admit
1178       4  0.999392           adult
1220       3  0.999696          advise
...      ...       ...             ...
46650      2  0.999295           viral
46698      1  0.111048           virus
46698      2  0.888907           virus
47344      2  0.403934            week
47344      3  0.595841            week

[259 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 4, 1, 3])

In [ ]:
def choose_topic(id2word,corpus,texts):#选择最合适的topic数：困惑度越低越好，一致性越高越好
    '''
    @description: 生成模型
    @param 
    @return: 生成主题数分别为1-15的LDA主题模型，并保存起来。
    '''
    #dictionary = corpora.Dictionary(data_lemmatized)
    id2word = Dictionary.load_from_text("id2word.txt")#加载生成的dictionary
    corpus = corpora.MmCorpus('ths_corpuse.mm')  # 加载
    texts = data_lemmatized
    coherence_values = []
    perplexity_values = []
    model_list = []
    for i in range(2,20,2):
        print('目前的topic个数:{}'.format(i))
        print('目前的数据量:{}'.format(len(texts)))
        temp = 'lda_{}_{}'.format(i,len(texts))
        model = gensim.models.ldamodel.LdaModel(corpus, num_topics=i, id2word=id2word, passes=20)
        file_path = './{}.model'.format(temp)
        model.save(file_path)

        
        coherence_model_lda = CoherenceModel(model=model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
        coherence_lda = coherence_model_lda.get_coherence()
        print('\nCoherence Score: ', coherence_lda)

        perplexity_values.append(model.log_perplexity(corpus))
        print('\nPerplexity: ', model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

        coherence_values.append(coherence_model_lda.get_coherence())
        
        print('------------------------------')
    return model_list, coherence_values, perplexity_values

In [ ]:
model_list, coherence_values, perplexity_values= choose_topic(id2word=id2word, corpus=corpus, texts=data_lemmatized)

#### 根据perplexity和coherence score选择最佳topic是15个 画五种药物topic分布图

In [ ]:
model=gensim.models.ldamodel.LdaModel.load("/data1/data_8t/ivermectin/newlda/lda_15_1994827.model")
doc_lda=model[corpus]
doc_topic = [a for a in model[corpus]]
print(len(doc_topic))
topic = []
p = []
for i in range(0,len(doc_topic)):
    c = []
    t = doc_topic[i]
    c.append([a[1] for a in t])
    m = max(c[0])
    n = min(c[0])
    if len(c[0])==1:
        topic.append(t[0][0])
        p.append(t[0][1])
        #data.loc[i, 'topic'] = t[0][0]
        #data.loc[i, 'p'] = t[0][1]
    elif m == n:
        topic.append(100)
        p.append(0)              
        #data.loc[i, 'topic'] = 100
        #data.loc[i, 'p'] = 0
    else:
        for j in range(0, len(t)):
            if m in t[j]:
                topic.append(t[j][0])
                p.append(t[j][1])
                #data.loc[i, 'topic'] = t[j][0]
                #data.loc[i, 'p'] = t[j][1]
                break


In [ ]:
from collections import Counter
result = Counter(topic)
print(result)
counts = pd.DataFrame(result.items(), columns=['label', 'counts'])
counts.sort_values("label",inplace=True, ascending = True)
counts=counts.reset_index(drop=True)
counts.to_csv('iver.csv',index=False) #五种药物 以ivermectin为例 最后总结成ldaplt3.csv


In [ ]:
data = pd.read_csv("/data1/data_8t/lwx/Drugtweets/药物分布/ivermectin.csv",low_memory=False, keep_default_na=False, lineterminator='\n')#原始文件
data['topic'] = topic
data['p'] = p
data=data[['full_text','topic','p']]


In [ ]:
for i in data[(data['topic']== 2) & (data['p']>0.7)]['full_text']: #看归属topic对应的推特
    print(i)
    print('--------------------------')

##### 绘图

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
ldaplt=pd.read_csv('ldaplt3.csv')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号
# x轴上坐标的个数，也是data列表里面的列表内的元素个数。4也可以用len(data[0])代替。
X = np.arange(15)
fig = plt.figure(figsize=(15,28),dpi=300)
ax = fig.add_axes([0.1,0.1,0.8,0.8])
plt.xticks(fontsize=25)
plt.yticks(fontsize=30)
colors= ["#91D1C2","#F39B7F","#E64B35","#7E6148","#8491B4","#4DBBD5","#3C5488","#8491B4","#91D1C2","#DC0000","#7E6148"]
'''
F1、F2、F3画柱状图，共三个，第一个参数可以认为是labels，第二个参数是各自的y坐标，如果我们想要
在某个柱状图下面设置x轴刻度值，就可以在该柱状图下面用tick_label设置。
'''
F5 = ax.barh(X + 0.00, ldaplt.iloc[:,5], color = colors[4], height = 0.18)
F4 = ax.barh(X + 0.18, ldaplt.iloc[:,4], color = colors[3], height = 0.18)
F3 = ax.barh(X + 0.36, ldaplt.iloc[:,3], color = colors[2], height = 0.18,tick_label=ldaplt['topics'])
F2 = ax.barh(X + 0.54, ldaplt.iloc[:,2], color = colors[1], height = 0.18)
F1 = ax.barh(X + 0.72, ldaplt.iloc[:,1], color = colors[0], height = 0.18)
#F6 = ax.bar(X + 0.75, ldaplt.iloc[:,6], color = colors[5], height = 0.15)
# 给柱状图设置图例，第一个参数是个元组，元组中每个元素代表一个柱状图，第二个参数是对应着的图例的名字。
ax.legend((F1,F2,F3,F4,F5),('Ivermectin','Hydroxychloroquine','Remdesivir','Zinc','Vitamin D'),fontsize=30)
#ax.legend((F1,F2,F3,F4,F5,F6),('Ivermectin','HCQ','Vit D','Remdesivir','Zinc','Total'),fontsize=20)
#plt.xticks(rotation=30)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.xlabel('Percentage of topics',fontsize=30)
plt.ylabel('Topics',fontsize=30)
#plt.title('Scatter',fontsize=30)
fig.savefig('shu2.tif', bbox_inches = 'tight',transparent=True,dpi=300)
plt.show()
